In [1]:
import pandas as pd
import numpy as np 
from datetime import timedelta, datetime 

from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objects as go

df = pd.read_excel("TFLBikes.xls", sheet_name="Data")
df["Day"] = pd.to_datetime(df["Day"])
df = df[["Day", "Number of Bicycle Hires"]]

df_test = df.iloc[2500:]
df = df.iloc[:2500]

df.head()

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df.tail()

First, lets just do an initial plot to try and gain a preliminary understanding of how we are going to create a model.

In [ ]:

fig = go.Figure(go.Scatter(
    x = df['Day'],
    y = df['Number of Bicycle Hires'],
    text = df['Day'].dt.strftime('%Y-%m-%d'),
    hovertemplate = '# of Bikes: %{y} <br>Date: %{text}<extra></extra>'
))
fig.update_layout(
    title = "Number of TFL Bikes per day"
)
fig.show()

We can observe a cyclical trend annually, but also we imagine that on weekends we would observe more uses as people have more free time. So let's now investigate the cyclical trends on an annually and a weekly basis.

First, we convert the day of the year to a value between 0 and 1 and then to an angle between 0 and 360 to plot with Plotly. We then plot this using a **polar plot**, allowing us to see the continuing 

In [ ]:
polar = df.groupby(df["Day"].dt.dayofyear.apply(lambda x: 360*x/366)).mean()


fig = go.Figure(data = [
    go.Scatterpolar(
        theta = df["Day"].dt.dayofyear.apply(lambda x: 360*x/366),
        r = df["Number of Bicycle Hires"],
        mode = 'markers',
        marker = dict(size=1),
        name = 'All Values',
    ),
    go.Scatterpolar(
        theta = polar.index.values,
        r = polar["Number of Bicycle Hires"],
        mode = 'lines',
        name = 'Mean',
    )])

fig.update_layout(
    title = "Repeated Yearly Trend",
    polar = dict(
        angularaxis = dict(
            ticktext = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'June', 'July', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
            tickvals = list(range(0,360, 30)),
            tickmode="array",
            rotation = 0
        )
    )
)
fig.show()

We observe a pattern which may seem familiar if you remember your polar co-ordinates from a maths class you may have took. It looks similar to a graph of the form $r = 1 - cos(\theta)$. Here's a little graph for you to play around with the values to convince yourself.

In [ ]:
# Create figure
fig = go.Figure()

# Add traces, one for each slider step
for step in np.arange(0, 2, 0.1):
    fig.add_trace(
        go.Scatterpolar(
            visible=False,
            name="𝜈 = " + str(step),
            theta = np.array(range(360)),
            r = [1 - step * np.cos(np.deg2rad(i)) for i in range(360)]
        ))

# Make 10th trace visible
fig.data[10].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to step: + {:.1f}".format(np.arange(0, 2, 0.1)[i])}],  # layout attribute
        label = "{0:.1f}".format(np.arange(0, 2, 0.1)[i])
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "r = 1 - ", "suffix": " cos(theta)"},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    polar = dict(
        radialaxis = dict(range=[0, 3])
    )
)

fig.show()

Now let's explore the cyclical weekly trend

In [ ]:
weekly = df.groupby(df["Day"].dt.dayofweek.apply(lambda x: 360*x/7)).mean()

weekly = weekly.append(weekly.iloc[0])

fig = go.Figure(data=[
    go.Scatterpolar(
        r = weekly['Number of Bicycle Hires'],
        theta = weekly.index,
        mode = 'lines',
        name = 'Average'
    ),
    go.Scatterpolar(
        theta = df["Day"].dt.dayofweek.apply(lambda x: 360*x/7),
        r = df["Number of Bicycle Hires"],
        mode = 'markers',
        marker = dict(size=3),
        name = 'All Values')
    ])
    
fig.update_layout(
    title = "Repeated Weekly Trend",
    polar = dict(
        angularaxis = dict(
            ticktext = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun'],
            tickvals = np.linspace(0,360,8)[:-1],
            tickmode="array",
            rotation = 0)
        )
)

We see a similar graph of what we saw for the cyclical annual graph except rotated slightly. To include this in our model we could apply a rotation by replacing the $\cos(\theta)$ term with a $\cos(\theta - \alpha)$. However, will use the identity $\cos(\theta - \alpha) = a \cos(\theta) + b \sin(\theta)$ to simplify).

## Beginning Modelling

We have identified some strong features that could be used to model the daily usage of bikes that are:

- $\cos(\frac{\text{day of year}}{366})$
- $\cos(\frac{\text{day of week}}{7})$
- $\sin(\frac{\text{day of week}}{7})$

First, let's see if the model works for the cyclic annual trend just using $\cos(\frac{\text{day of year}}{366})$ and a bias term. We set up a simple linear regression with a bias term


In [ ]:
df['PolarDay'] = df['Day'].dt.dayofyear.apply(lambda x: 360*x/366)

A = np.array([[1, np.cos(i)] for i in df['PolarDay'].apply(lambda x: np.deg2rad(x)).values])

b = df[['Number of Bicycle Hires']].values

x = np.linalg.lstsq(A,b)[0]

bhat = np.dot(A, x)

fig = go.Figure(data=[
    go.Scatterpolar(
        theta = df["Day"].dt.dayofyear.apply(lambda x: 360*x/366),
        r = df["Number of Bicycle Hires"],
        mode = 'markers',
        marker = dict(size=1),
        name = 'All Values',
    ),
    go.Scatterpolar(
        r = [b[0] for b in bhat],
        theta = df['PolarDay'],
        mode = 'lines',
        name = 'Predicted'
    ),
    go.Scatterpolar(
        theta = polar.index.values,
        r = polar["Number of Bicycle Hires"],
        mode = 'lines',
        name = 'Mean',
    )]
)
fig.update_layout(
    title = "Repeated Yearly Trend",
    polar = dict(
        angularaxis = dict(
            ticktext = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'June', 'July', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
            tickvals = list(range(0,360, 30)),
            tickmode="array",
            rotation = 0
        )
    )
)
fig.show()

We see the model performs okay, but is yet to compensate for the weekly fluctuations. So now let's try the weekly fluctuations.

In [ ]:
df['PolarWeekday'] = df['Day'].dt.dayofweek.apply(lambda x: 360*x/7)

A = np.array([[1, np.cos(i), np.sin(i)] for i in df['PolarWeekday'].apply(lambda x: np.deg2rad(x)).values])

b = df[['Number of Bicycle Hires']].values

x = np.linalg.lstsq(A,b)[0]

bhat = np.dot(A, x)

fig = go.Figure(data=[
    go.Scatterpolar(
        r = df['Number of Bicycle Hires'],
        theta = df['PolarWeekday'],
        mode = 'markers',
        name = 'actual'
    ),
    go.Scatterpolar(
        r = weekly.append(weekly.iloc[0])['Number of Bicycle Hires'],
        theta = weekly.append(weekly.iloc[0]).index,
        mode = 'lines',
        name = 'Average'
    ),
    go.Scatterpolar(
        r = pd.DataFrame(bhat)[0],
        theta = df['PolarWeekday'],
        mode = 'lines',
        name = 'predicted'
    )]
)

fig.update_layout(
    title = "Repeated Weekly Trend",
    polar = dict(
        angularaxis = dict(
            ticktext = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun'],
            tickvals = np.linspace(0,360,8)[:-1],
            tickmode="array",
            rotation = 0)
        )
)

We see this slightly more (1 additional term) model is able to cope with the variety of weekdays (roughly). I.e. Significantly lower on weekends than weekdays. Now let's combine the two models to see how we do.

In [ ]:
import numpy as np 

weekly = df.groupby(df["Day"].dt.dayofweek.apply(lambda x: 360*x/7)).mean()

A = np.array([[1, np.cos(i), np.sin(i)] for i in weekly.reset_index()['Day'].apply(lambda x: np.deg2rad(x)).values])

b = weekly[['Number of Bicycle Hires']].values

x = np.linalg.lstsq(A,b)[0]

bhat = np.dot(A, x)

fig = go.Figure(data=[
    go.Scatterpolar(
        r = weekly.append(weekly.iloc[0])['Number of Bicycle Hires'],
        theta = weekly.append(weekly.iloc[0]),
        mode = 'lines',
    ),
    go.Scatterpolar(
        r = [x[0][0] + x[1][0]*np.cos(np.deg2rad(i)) + x[2][0]*np.sin(np.deg2rad(i)) for i in range(360)],
        theta = list(range(360)),
        mode = 'lines',
        name = 'predicted'
    )]
)
fig.update_layout(
    title = "Repeated Yearly Trend"
)
fig.show()

In [ ]:
def linRegModel(features, df_train, df_test = None, verbose = True):
    
    # Adding Polar Features
    df_train = createPolarFeatures(df_train)

    # Creating Feature Matrix
    A = features(df_train)

    # Creating Target Matrix
    b = df_train[['Number of Bicycle Hires']].values

    # Creating Model
    model = sm.OLS(b, A)

    # Fitting Model
    results = model.fit()

    # Printing Model Results
    if verbose:
        print(results.summary())

    if df_test is None:

        return insertResults(results, df_train, features)
    
    else:
        df_test = createPolarFeatures(df_test)

        A_test = features(df_test)

        return insertResults(results, df_test, features)

def simpleFeatures(df):

    A = pd.concat([
        df['PolarDay'].apply(np.deg2rad).apply(np.cos),
        df['PolarWeekday'].apply(np.deg2rad).apply(np.cos),
        df['PolarWeekday'].apply(np.deg2rad).apply(np.sin),
        df['PolarDay'].apply(lambda x: 1),
    ], axis=1).values

    return A


def insertResults(results, df, features):

    A = features(df)

    # Calculating Upper and Lower Predictions
    prstd, iv_l, iv_u = wls_prediction_std(results, exog = A,  alpha=0.05)

    # Predicting the Model
    df['Predicted'] = results.predict(exog = A)

    # Calculating the Error for each entry
    if 'Number of Bicycle Hires' in df.columns:
        df['Error'] = df['Predicted'] - df['Number of Bicycle Hires']

    # Inserting the CI
    df['CI_Lower'] = iv_l
    df['CI_Upper'] = iv_u

    return df

def createPolarFeatures(df):
    df['PolarDay'] = df['Day'].dt.dayofyear.apply(lambda x: 360*x/366)
    df['PolarWeekday'] = df['Day'].dt.dayofweek.apply(lambda x: 360*x/7)
    return df


df = linRegModel(simpleFeatures, df)

We got an okay training RMSE, however let's see if we can do a little better by observing a global (non-cyclical) trend to the data. To do this, let's plot a rolling annual average to gain some sense of the data that is immune to any seasonal trends.

In [ ]:
fig = go.Figure(data = [
    go.Scatter(
    x = df['Day'],
    y = df['Number of Bicycle Hires'],
    text = df['Day'].dt.strftime('%Y-%m-%d'),
    hovertemplate = '# of Bikes: %{y} <br>Date: %{text}<extra></extra>',
    name = 'All data'
    ),
    go.Scatter(
    x = df['Day'],
    y = df['Number of Bicycle Hires'].rolling(365).mean(),
    text = df['Day'].dt.strftime('%Y-%m-%d'),
    hovertemplate = '# of Bikes: %{y} <br>Date: %{text}<extra></extra>',
    name = 'Rolling Average (365 days)'
    ),
]
    )
fig.update_layout(
    title = "Number of TFL Bikes per day"
)
fig.show()

We observe a general shallow increment over time. As we've focused on keeping this a simple linear model, let's just introduce a linear increasing term for each day and see how that does!

In [ ]:
def linearFeatures(df):

    A = pd.concat([
        df['PolarDay'].apply(np.deg2rad).apply(np.cos),
        df['PolarWeekday'].apply(np.deg2rad).apply(np.cos),
        df['PolarWeekday'].apply(np.deg2rad).apply(np.sin),
        df['PolarDay'].apply(lambda x: 1),
        pd.DataFrame(df.index.values, index = df.index)
    ], axis=1).values

    return A

df = linRegModel(linearFeatures, df)

We managed to decrease our RMSE by nearly 1000, which is an excellent result! Now let's plot the results with confidence intervals

In [ ]:
def linegraphFactory(df, true_vals = True, predicted = True, CI = True, suffix = ""):
    
    plots = []

    if true_vals:
        plots.append(go.Scatter(
            x = df['Day'],
            y = df['Number of Bicycle Hires'],
            text = df['Day'].dt.strftime('%Y-%m-%d'),
            hovertemplate = '# of Bikes: %{y} <br>Date: %{text}<extra></extra>',
            line=dict(width=1, color = "rgba(234,53,70)"),
            name = "Actual"+suffix
            )
        )
         
    if predicted:
        plots.append(
            go.Scatter(
                x = df['Day'],
                y = df['Predicted'],
                text = df['Day'].dt.strftime('%Y-%m-%d'),
                hovertemplate = '# of Bikes: %{y} <br>Date: %{text}<extra></extra>',
                line=dict(width=1, color = "rgba(248,102,36)"),
                name = "Predicted"+suffix,
                )
        )

    if CI:
        plots.append(go.Scatter(
            x = pd.DataFrame(np.concatenate((df['Day'].values, df['Day'].values[::-1])))[0],
            y = pd.concat((df['CI_Lower'], df['CI_Upper'].iloc[::-1])),
            line=dict(width=1, dash='dash', color = "rgba(249,200,14,0.5)"),
            mode = "lines",
            name = "CI"+suffix,
            fill = "toself",
            fillcolor = "rgba(249,200,14,0.5)"
            )
        )

    return plots

train_plots = linegraphFactory(df, suffix = "train")

fig = go.Figure(data = train_plots
)
fig.update_layout(
    title = "TRAIN MSE = {:.5g} Bikes".format(df['Error'].apply(lambda x: x**2).mean() ** 0.5)
)
fig.show()

We observe that the model is coping with the trends pretty well but let's investigating where it is failing (i.e the true value lies outside of the CI)

In [ ]:
cond1 = df['CI_Lower'] > df['Number of Bicycle Hires']
cond2 = df['CI_Upper'] < df['Number of Bicycle Hires']

print('Outliers per weekday')
print(df[(cond1) | (cond2)]['Day'].dt.dayofweek.value_counts())

print('Outliers per month')
print(df[(cond1) | (cond2)]['Day'].dt.month.value_counts())

print('Outliers per year')
print(df[(cond1) | (cond2)]['Day'].dt.year.value_counts())

print('Outliers per month-year')
print(df[(cond1) | (cond2)]['Day'].apply(lambda x: "{0}-{1}".format(x.month, x.year)).value_counts().head(10))

print('Outliers per weekday-month')
print(df[(cond1) | (cond2)]['Day'].apply(lambda x: "{0}-{1}".format(x.dayofweek, x.month)).value_counts().head(10))

We see that 32 of the occurances are from the beginning of the dataset (30 from 8-2010 and 2 from 7-2010). We will omit these from the dataset in the future. In fact, we will remove all dates before 2012 

Saturdays and Sundays (combinging to 52 outliers) still proved much trickier for the model to predict - particularly in August (which had 23 outliers). So let's do some further investigation as to why this might be occuring - a preiliminary guess would be that during August and on weekends the usage is a lot more varied particularly as more hobbyists would use the bikes. This would mean that usage is probably more dependent on the weather etc - variables that we have neglected to consider.

Let's explore the training data by using boxplots.

In [ ]:
df['Month'] = df['Day'].dt.month_name()
df['MonthNumber'] = df['Day'].dt.month

N = 12 
c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, N)]
print(df.groupby('Month')['Number of Bicycle Hires'].std().sort_values(ascending=False))
fig = px.box(df.sort_values("MonthNumber"), x="Month", y='Number of Bicycle Hires', color = "Month",color_discrete_sequence=px.colors.qualitative.Dark24)
fig.show()

We can observe the largest interquartile range of approximately 16k in August. This is reassuring as it indicates there isn't an underlying flaw in the model we've created causing it to predict during August.

In [ ]:
df['Weekday'] = df['Day'].dt.day_name() 
df['WeekdayNumber'] = df['Day'].dt.dayofweek


N = 12 
c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, N)]
print(df.groupby('Weekday')['Number of Bicycle Hires'].std().sort_values(ascending=False))
fig = px.box(df.sort_values('WeekdayNumber'), x="Weekday", y='Number of Bicycle Hires', color = "Weekday",color_discrete_sequence=px.colors.qualitative.Dark24)
fig.show()

Similarly, we see a larger variation over weekends, again indicating that weekends will be harder to predict.

Let's also investigate areas where the model failed the most or the data is most anomalous. As this is real world data we can investigate it and try and find reasons for this anomalous behaviour.

In [ ]:
outliers = pd.concat([
    df[['Day','Weekday', 'Number of Bicycle Hires', 'Predicted', 'Error']].sort_values('Error').head(5),
    df.iloc[33:][['Day','Weekday', 'Number of Bicycle Hires', 'Predicted', 'Error']].sort_values('Error').tail(5),
])
outliers 

We see the the top 5 Overshooting and Undershooting occurances in the model. It is quite fun that we can google what happened in London on these days - including Tube Strikes, the Olympics Speed Walking and some very sunny and rainy days (imagine that in a British Summer!) 

# Undershooting
- 2015-07-09 - Tube Strike Action - https://www.itv.com/news/london/story/2015-07-09/strike-brings-londons-tube-network-to-a-standstill/
- 2015-08-06 - Tube Strike Action - https://www.bbc.co.uk/news/live/uk-england-london-33674627
- 2016-12-25 - Planned Strike Action/Christmas Day - https://en.wikipedia.org/wiki/London_Underground_strikes
- 2017-01-09 - Strike Action -https://www.itv.com/news/london/2017-01-09/tube-strike-tfl-advise-passengers-to-complete-their-journey-by-6pm
- 2017-04-09 - Really nice day - https://www.itv.com/news/2017-04-09/its-official-uk-enjoys-hottest-day-of-2017


# Overshooting
- 2014-08-25 - Rainy Bank Holiday Monday - https://metro.co.uk/2014/08/25/notting-hill-carnival-2014-the-rain-doesnt-dampen-the-crowd-on-bank-holiday-monday-4845458/
- 2012-06-11 - Lots of roads shut off because of speed-walk in London 2012 Olympics - https://www.standard.co.uk/news/transport/800000-extra-train-seats-laid-on-for-penultimate-day-of-the-games-8034368.html
- 2012-04-09 - Bank Holiday
- 2011-06-12 - Possibly just extremely rainy - particularly for June
- 2011-06-05 - Again possibly extremely rainy

Okay so lets remove these points from the data as well as the earlier data and retrain our model!

In [ ]:
removing = np.concatenate([
                outliers.index.values,
                df[df['Day'].dt.year <= 2011].index.values
                ])

df_drop = linRegModel(linearFeatures, df.drop(removing), verbose = False)

train_drop_plots = linegraphFactory(df_drop)

fig = go.Figure(data = train_drop_plots
)
fig.update_layout(
    title = "TRAIN MSE = {:.5g} Bikes".format(df_drop['Error'].apply(lambda x: x**2).mean() ** 0.5)
)
fig.show()

As expected the train MSE has gone down, but hopefully this technique also helps reduce andy overfitting effects on the test set.

In [ ]:
df_test = linRegModel(linearFeatures, df_drop, df_test = df_test, verbose = False)

test_plots = linegraphFactory(df_test, suffix = "_test")

fig = go.Figure(data = train_drop_plots+ test_plots
)
fig.update_layout(shapes=[
    dict(
      type= 'line',
      yref= 'paper', y0= 0, y1= 1,
      xref= 'x', x0= df_test['Day'].min(), x1= df_test['Day'].min(),
      line = {'color': 'black','width': 1, 'dash': 'dot',}
    )],
    title = "TEST MSE = {:.5g} Bikes".format(df_test['Error'].apply(lambda x: x**2).mean() ** 0.5))
fig.add_trace(go.Scatter(
    x=[df_test['Day'].min()],
    y=[3_000],
    mode="text",
    text=[f"{df_test['Day'].min().date()} "],
    textposition='middle left',
    name = ""
))
fig.show()

Our test MSE is a little higher than our training result, but it appears sensible and is quite close so this is quite a promising result. However, even better than that - as our model independent or previous results (like an LSTM wouldn't be) - our error doesn't seem to increase as time continues.

In [ ]:
fig = go.Figure(
    data = go.Scatter(
        x = df_test['Day'],
        y = pd.concat([df, df_test])['Error'].rolling(365).mean().iloc[-len(df_test):],
        mode = 'lines'
    )
)
fig.update_layout(title = 'Test Rolling Average (365 Days) Error')
fig.show()

Now to wrap things up, let's try and predict to the present day even if we can't evaluate the results. 

In [ ]:
future_df = pd.DataFrame(
            pd.Series([df_test['Day'].max() + timedelta(days=i) for i in range((datetime.today() - df_test['Day'].max()).days)], 
            name = "Day")
            )

future_df.index += df_test.index.max() + 1

future_df = linRegModel(linearFeatures, df_drop, df_test = future_df, verbose = False)

future_plots = linegraphFactory(future_df, true_vals=False, suffix = "_future")

fig = go.Figure(data = train_drop_plots+ test_plots + future_plots
)
fig.update_layout(shapes=[
    dict(
      type= 'line',
      yref= 'paper', y0= 0, y1= 1,
      xref= 'x', x0= df_test['Day'].min(), x1= df_test['Day'].min(),
      line = {'color': 'black','width': 1, 'dash': 'dot',}
    ),
    dict(
      type= 'line',
      yref= 'paper', y0= 0, y1= 1,
      xref= 'x', x0= future_df['Day'].min(), x1= future_df['Day'].min(),
      line = {'color': 'black','width': 1, 'dash': 'dot',}
    ),

])
fig.add_trace(go.Scatter(
    x=[df_test['Day'].min(), future_df['Day'].min()],
    y=[3_000, 3_000],
    mode="text",
    text=[f"{df_test['Day'].min().date()} ", f"{future_df['Day'].min().date()} "],
    textposition='middle left',
    name = ""
))

fig.update_layout(title = 'Prediction to the Present Day')

fig.show()

One other bonues of this method is we have an entirely linear model - which makes it very interpretable. So say for example, you were working on a data science project at TFL and had to report to the stakeholders about how your model predicted the number of bikes used per day. In this scenario you could present them an easy to notate mathematical equation that would be simple to understand (or at least simpler than a black box model like a neural net).

I hoped you enjoyed reading this notebook, and if you have any comments please let me know!